# COVID-19 Daily Report All Regions
### Generates standalone interactive HTML &/or deployment on web server (local host or cloud provider)

## Author: Prasanna Badami

### <font size = '4'> Note:
<font size = '4'> This source code,
- <font size = '3'> runs in Jupyter notebook with environment meeting all libraries used in this source code.
- <font size = '3'> generates standalone HTML output in the same directory as this IPYNB file. User can modify the code to generate the output in Jupyter notebook.
- <font size = '3'> needs internet connection, it downloads an input file from CSSEGISandData repository on GitHub. Internet charges may apply. <br>
    
  
<font size = '3'>To get the output in web browser similar to standalone interactive HTML without exposing code, you need to use Bokeh server deployment, which deploys output of this source code on local system or cloud provider. This is done by adding panel.servable() in source file & launching the app using 'panel serve sourcefile.ipynb' at command prompt 

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from numpy import nan

In [ ]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Div, HoverTool, NumeralTickFormatter, Select, CustomJS, DataTable, TableColumn
from bokeh.resources import INLINE

In [ ]:
import panel as pn
pn.extension()

In [ ]:
date_on_file_today = datetime.today().strftime('%m-%d-%Y')

In [ ]:
date_on_file_yesterday =  (datetime.today() - timedelta(days = 1)).strftime('%m-%d-%Y')

In [ ]:
date_on_file_day_b4_yesterday = (datetime.today() - timedelta(days = 2)).strftime('%m-%d-%Y')

In [ ]:
date_on_save_html = 'Today'

In [ ]:
# First try to get file with today's date, if not available,
# try with yesterday's date, if not available,
# try with day before yesterday's date, if not available 
# assign empty dataframe to df_daily_report, this dataframe is for checked contents in next cell.

In [ ]:
try:
    df_daily_report = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+ date_on_file_today + '.csv')
    date_on_save_html = date_on_file_today
    print('Selected %s.csv' % date_on_file_today)
except:
    try:
        df_daily_report = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+ date_on_file_yesterday + '.csv')
        date_on_save_html = date_on_file_yesterday
        print('Selected %s.csv' % date_on_file_yesterday)
    except:
        try:
            df_daily_report = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+ date_on_file_day_b4_yesterday + '.csv')
            date_on_save_html = date_on_file_day_b4_yesterday
            print('Selected %s.csv' % date_on_file_day_b4_yesterday)
        except:
            df_daily_report = pd.DataFrame()
            print('HTTPError: HTTP Error 404: File Not Found')

In [ ]:
# df_daily_report

In [ ]:
pn_dashboard = pn.Column(Div(text = 'HTTPError: HTTP Error 404: Input file for ' + str(date_on_file_today) + ' or ' \
                             +  str(date_on_file_yesterday) + ' or ' + str(date_on_file_day_b4_yesterday) + ' not found on GitHub', width = 925))
if (df_daily_report.empty == True):
#     Disable GUI & all analysis here as file is not found.
    print('Since file is not found on server, dataframe is assigned to empty. Not proceeding with further analysis')
else:
    # x = 'Categories', y = 'Country_Province_State', source = master_source
    
#     df_drfc: dataframe_DailyReportFilteredCopy, we are interested in columns filtered
#     df_drfc is used to create dictionary, which in turn is used to create ColumnDataSource for plots.
#     Logic here is: to have a categorical column 'categories' & columns for every country, country_province & country_province_admin2 corresponding to each category.
#     default is added for displaying first plot to display when standalone HTML output is opened.. here my home country & province are selected as default.
#     color column is added to fill bar color for each 'categories' in bar plot
    df_drfc = df_daily_report.filter(['Admin2', 'Province_State', 'Country_Region', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Incident_Rate', 'Case_Fatality_Ratio']).copy()

    ddict = dict({'categories': ['Confirmed', 'Recovered', 'Deaths', 'Active', 'Incident_Rate', 'Case_Fatality_Ratio']})
    for i, j, k, l, m, n, o, p, q in zip(df_drfc.Country_Region.tolist(), df_drfc.Province_State.tolist(), df_drfc.Admin2.tolist(),
                                         df_drfc.Confirmed.tolist(), df_drfc.Recovered.tolist(), df_drfc.Deaths.tolist(), df_drfc.Active.tolist(),
                                         df_drfc.Incident_Rate.round().tolist(), df_drfc.Case_Fatality_Ratio.round(2).tolist()
                                        ):
        ddict.update({str(i)+'_'+str(j)+'_'+ str(k): [l, m, n, o, p, q]})
        if(str(i)+'_'+str(j)+'_'+ str(k) == 'India_Karnataka_nan'):
            ddict.update({'default': [l, m, n, o, p, q]})
    ddict.update({'color' : ['royalblue', 'lawngreen', 'red', 'orange', 'skyblue', 'blue']})
    
    dsource = ColumnDataSource(ddict)

#     for select box provinces based on country
    provinces_df = df_daily_report.pivot(columns = 'Country_Region', values = 'Province_State')
    
#     for select box admin2 based on province
    admin2_df = df_daily_report.pivot(columns = 'Province_State', values = 'Admin2')
    
#     filter out all 'nan' in 'admin2 dataframe', because 'Province_State' do have 'nan' in 'Admin2'
    admin2_df.drop(columns = nan, inplace = True)
    
#     for select box country
    countries_region_list = df_daily_report['Country_Region'].unique().tolist()

#     last update is used in plot title & footer
    last_update = df_daily_report['Last_Update'][0]
    
    hover_tool2 = HoverTool(tooltips = [('Metric', '@categories'), ('Cases', '@default{0,}')])

    tp = figure(x_range = ['Confirmed', 'Recovered', 'Deaths', 'Active'], width = 930, height = 600, tools = [hover_tool2], toolbar_location = None)

    tp.vbar(x='categories', top= 'default', width = 0.4, line_width = 5, 
            fill_color = 'color', line_color = 'white', source = dsource)
   
#     tp.xaxis.major_label_orientation = 45

    tp.xaxis.major_label_text_font_style = 'bold'
    
    tp.y_range.start = 0
    tp.yaxis.formatter = NumeralTickFormatter(format = 'a')
    tp.yaxis.axis_label = 'COVID-19 Cases'
    tp.yaxis.axis_label_text_font_style = 'bold'
    tp.yaxis.major_label_text_font_style = 'bold'
    
    tp.title.text = 'Daily Report:' + ' | ' + 'India_Karnataka' + ' | ' + 'Incident Rate: ' + str(round(dsource.data['default'][4])) + \
                    ' | ' + 'Case Fatality Ratio: ' + str(round(dsource.data['default'][5], 2)) + ' | ' + 'Last update: ' + str(last_update)

#     datatable used to display plot data based on each selection (Counrty, Province & Admin2) by user
    tablecolumns = [TableColumn(field = 'categories', title = 'Category'), TableColumn(field = 'default', title = 'COVID-19 Cases')]

    datatable = DataTable(source = dsource, columns = tablecolumns, width = 300, height = 300)
    
#     Select box for Country
    s_countries = Select(title = 'Country',
                         options = countries_region_list, value = 'India',
                         width = 250,
                        )
#     Select box for Provinces
    s_provinces = Select(title = 'Province',
                         options = provinces_df['India'].dropna().tolist(), value = 'Karnataka',
                         width = 250
                        )
#     Select box for Admin2
    s_admin2 = Select(title = 'Admin2',
                      options = [],
                      width = 250
                     )
#     footer for plot
    footer = Div(text = """Graphic: Prasanna Badami 
                        <div>Source: Center for Systems Science & Engineering, Johns Hopkins University</div> 
                        <div>Last update: {} </div> 
                        <div style = "font-size:12px"> Note:</div>
                        <div style = "font-size:12px"><i><t>Incident rate is calculated as confirmed cases per 100,000 population.</i></div>
                        <div style = "font-size:12px"><i> Case fatality ratio is the ratio of deaths to confirmed cases.</i></div>
                        """.format(last_update),  
                     width = 925,
                     background = 'white',
    #                  style={'font-family': 'Helvetica', 
    #                         'font-style' : 'italic', 
    #                         'font-size' : '12px', 
    #                         'color': 'black',
    #                         'font-weight': 'bold'
    #                        },
                    )
    
#     Custom JavaScript for select box country
    call_country = CustomJS(args = {'s_states' : s_provinces, 's_counties': s_admin2,
                                    'states': ColumnDataSource(provinces_df),
                                    'counties': ColumnDataSource(admin2_df),
                                    'source': dsource, 'dtable': datatable,
                                    'lastupdate' : last_update,
                                       'pt': tp
                                      },
                            code = """

                                   // Javascript code

                                   var category
                                   var tcategory = cb_obj.value
                                   var province = 'nan'
                                   var admin = 'nan'

                                      // cb_obj.value contains value of s_countries: name of a country
                                      // Filter out NaN values
                                    var foptions1 = states.data[cb_obj.value].filter(function(value, index, self){return value != 'NaN'})
                                    var poptions = foptions1.filter(function(value, index, self){return self.indexOf(value) == index})
                                    s_states.options = poptions

                                   if(s_states.options[0] == undefined){
                                        s_counties.options = poptions
                                        admin = 'nan'
                                        province = 'nan'
                                   }
                                   else{
                                        province = s_states.options[0]
                                        var foptions2 = counties.data[s_states.options[0]].filter(function(value, index, self){return value != 'NaN'})
                                        var coptions = foptions2.filter(function(value, index, self){return self.indexOf(value) == index})
                                        s_counties.options = coptions

                                        if (s_counties.options[0] == undefined){
                                            admin = 'nan'
                                        }
                                        else{
                                            admin = s_counties.options[0]
                                            s_states.value = s_states.options[0]                      
                                        }

                                   }
                                    category = tcategory.concat('_', province, '_', admin)

                                      // if the province for country is null then counties will also be null, if not null assign counties in Selectbox admin2
                                      // if counties are null, assign that to Selectbox admin2, this will most probably be updated to disable Selectbox itself

                                     source.data['default'] = source.data[category]
                                     source.change.emit()
                                     dtable.columns[1].field = category
                                     dtable.change.emit()
                                     if(province != 'nan' && admin != 'nan'){
                                         pt.title.text = 'Daily Report:' + ' | ' + category + ' | ' 
                                                         + 'Incident Rate: ' + Math.round(source.data[category][4]) + ' | '
                                                         + 'Case Fatality Ratio: ' + parseFloat(source.data[category][5]).toFixed(2)
                                                         + ' | ' + 'Last update: ' + lastupdate

                                     }
                                     else if(province == 'nan' && admin == 'nan'){
                                         pt.title.text = 'Daily Report:' + ' | ' + tcategory + ' | ' 
                                                         + 'Incident Rate: ' + Math.round(source.data[category][4]) + ' | '
                                                         + 'Case Fatality Ratio: ' + parseFloat(source.data[category][5]).toFixed(2)
                                                         + ' | ' + 'Last update: ' + lastupdate
                                     }
                                     else if(province != 'nan'){
                                         pt.title.text = 'Daily Report:' + ' | ' + tcategory + ' _ ' + province + ' | '
                                                         + 'Incident Rate: ' + Math.round(source.data[category][4]) + ' | '
                                                         + 'Case Fatality Ratio: ' + parseFloat(source.data[category][5]).toFixed(2)
                                                         + ' | ' + 'Last update: ' + lastupdate
                                     }
                                     else{
                                      // do nothing
                                     }
                                     pt.visible = true

                                   """
                               )

#     calls Custom JavaScript call_country whenever value is changed in select box country
    s_countries.js_on_change('value', call_country)

#     Custom JavaScript for select box provinces
    call_provinces = CustomJS(args = {'countries': s_countries, 's_states' : s_provinces, 's_counties': s_admin2,
                                      'counties': ColumnDataSource(admin2_df),
                                     'source': dsource, 'dtable': datatable,
                                      'lastupdate' : last_update, 
                                       'pt': tp
                                      }, 
                               code = """
                                     if (cb_obj.value != null){
                                      var category
                                      var admin
                                      var province = s_states.value
                                      var tcategory = countries.value
                                      var coptions = counties.data[cb_obj.value].filter(function(value, index, self){return value != 'NaN'})
                                      var ucoptions = coptions.filter(function(value, index, self){return self.indexOf(value) == index})

                                      s_counties.options = ucoptions

                                      if(s_counties.options[0] == undefined){
                                            s_counties.value = null
                                            admin = 'nan'
                                      }
                                      else{
                                          s_counties.value = s_counties.options[0]
                                          admin = s_counties.options[0]
                                      }

                                      category = tcategory.concat('_',province, '_', admin)
                                      // if the province for country is null then counties will also be null, if not null assign counties in Selectbox admin2
                                      // if counties are null, assign that to Selectbox admin2, this will most probably be updated to disable Selectbox itself

                                     source.data['default'] = source.data[category]
                                     source.change.emit()
                                     dtable.columns[1].field = category
                                     dtable.change.emit()
                                     if (admin == 'nan'){
                                         pt.title.text = 'Daily Report:' + ' | ' + tcategory + '_' + province + ' | ' 
                                                         + 'Incident Rate: ' + Math.round(source.data[category][4]) + ' | '
                                                         + 'Case Fatality Ratio: ' + parseFloat(source.data[category][5]).toFixed(2)
                                                         + ' | ' + 'Last update: ' + lastupdate
                                     }
                                     else{
                                         pt.title.text = 'Daily Report:' + ' | ' + category + ' | ' 
                                                         + 'Incident Rate: ' + Math.round(source.data[category][4]) + ' | '
                                                         + 'Case Fatality Ratio: ' + parseFloat(source.data[category][5]).toFixed(2)
                                                         + ' | ' + 'Last update: ' + lastupdate
                                     }
                                     pt.visible = true
                                     }
                                    """
                               )
#     calls Custom JavaScript call_provinces whenever value is changed in select box province
    s_provinces.js_on_change('value', call_provinces)
    
#     Custom JavaScript for select box admin2
    call_admin2 = CustomJS(args = {'countries': s_countries, 's_states' : s_provinces, 's_counties': s_admin2,
                                    'source': dsource, 'dtable': datatable,
                                    'lastupdate' : last_update,
                                    'pt': tp
                                  }, 
                           code = """
                                 if (cb_obj.value != null){
                                     var category
                                     var admin = cb_obj.value
                                     var province = s_states.value
                                     var tcategory = countries.value

                                     category = tcategory.concat('_', province, '_', admin)
                                  // alert(category) 
                                  // if the province for country is null then counties will also be null, if not null assign counties in Selectbox admin2
                                  // if counties are null, assign that to Selectbox admin2, this will most probably be updated to disable Selectbox itself

                                     source.data['default'] = source.data[category]
                                     source.change.emit()
                                     dtable.columns[1].field = category
                                     dtable.change.emit()
                                     pt.title.text = 'Daily Report:' + ' | ' + category + ' | ' 
                                                     + 'Incident Rate: ' + Math.round(source.data[category][4]) + ' | '
                                                     + 'Case Fatality Ratio: ' + parseFloat(source.data[category][5]).toFixed(2)
                                                     + ' | ' + 'Last update: ' + lastupdate
                                     pt.xaxis.axis_label = 'category'
                                     pt.visible = true
                                     }

                                  """
                            )
#     calls Custom JavaScript call_admin2 whenever value is changed in select box admin2
    s_admin2.js_on_change('value', call_admin2)
    
#     Arrange JavaScript in order for display
    pn_dashboard = pn.Column(pn.Row(s_countries, s_provinces, s_admin2), pn.Column(pn.Row(tp, datatable)), footer, background= 'aliceblue')
    
# To standalone interactive HTML,
pn_dashboard.save('output\COVID-19_Daily_Report_JS_' + str(date_on_save_html)+'.html', 
                      resources = INLINE, 
                      embed = True, 
                      title = 'Daily Report'
                     )
# For server deployment
pn_dashboard.servable(title = 'COVID-19_Daily_Report_All_Regions_'+ str(date_on_save_html))